In [ ]:
import utils
import numpy as np
import torch
import random
import torch.nn.functional as F
from torch import nn
import pickle
import scipy
import pandas as pd

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

TEST_BATCH=1024

NUM_CLASSES=2

NUM_WORKERS=0
PIN_MEMORY=True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class ANN(nn.Module):
    def __init__(self,in_features,num_classes):
        super().__init__()
        self.fc1=nn.Linear(in_features=in_features,out_features=512)
        self.fc2=nn.Linear(in_features=512,out_features=256)
        self.fc3=nn.Linear(in_features=256,out_features=num_classes)
    def forward(self,x):
        pred=F.relu(self.fc1(x))
        pred=F.relu(self.fc2(pred))
        pred=self.fc3(pred)
        return pred

In [ ]:
class SparseDataset(torch.utils.data.Dataset):
    def __init__(self,data,labels):
        self.data=data
        self.labels=labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,index):
        return self.data[index].toarray()[0],self.labels[index]

In [ ]:
def preproc_features(df):
    cols=df.columns
    cols=[col for col in cols if col not in utils.NO_1HOT_COLS]
    with open('./models/preproc.sav', 'rb') as file:
        preproc=pickle.load(file)
    enc=preproc['enc']
    df1=enc.transform(df[cols])
    df2=df.drop(cols,axis=1)
    df2=(df2-preproc['mean'])/preproc['std']
    df2=scipy.sparse.csr_matrix(df2.values)
    sp=scipy.sparse.hstack([df1,df2])
    return sp.astype(np.float32)

In [ ]:
def load_data():
    features=utils.load_data(input_path="./input_data/input_data.csv",separate=False)
    features=preproc_features(features)
    test_loader=torch.utils.data.DataLoader(
        SparseDataset(features,np.zeros((features.shape[0],))),
        batch_size=TEST_BATCH,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    return test_loader

In [ ]:
def test(test_loader, model):
    outputs=[]
    model=model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data).argmax(axis=1)
            output=list(map(lambda x: x.item(),list(output.to('cpu'))))
            outputs.extend(output)
    return outputs

In [ ]:
def main():
    with open('./models/model.pth', 'rb') as file:
        model=pickle.load(file)
    test_loader=load_data()
    model=model.to(device)
    outputs=test(test_loader,model)
    df=pd.DataFrame(data=outputs,columns=['Delay'])
    df.to_csv('./outputs.csv',index=False)
    print(df.head())

In [ ]:
if __name__ == "__main__":
    main()